In [1]:
import geopandas as gpd
import pandas as pd
import pandana as pdn
import osmnx as ox
import numpy as np

def get_path(source ,feature, mode='caminata'):
    return f'/app/data/{source}/red_escenario_futuro/{mode}/{feature}_conce'
def get_input_nodes():
    return get_path('input', 'nodes')
def get_input_edges():
    return get_path('input', 'edges')
def get_output_nodes():
    return get_path('output', 'nodes')
def get_output_edges():
    return get_path('output', 'edges')

In [2]:
nodes_df = gpd.read_file(get_output_nodes())
edges_df = gpd.read_file(get_output_edges())

In [8]:
nodes = pd.DataFrame(
    {
        'osmid': nodes_df['osmid'].astype(int),
        'lat' : nodes_df.geometry.y.astype(float),
        'lon' : nodes_df.geometry.x.astype(float),
        'y' : nodes_df.geometry.y.astype(float),
        'x' : nodes_df.geometry.x.astype(float),
    }
)
nodes['id'] = nodes['osmid'].values

nodes = gpd.GeoDataFrame(data=nodes, geometry=nodes_df.geometry)
nodes.set_index('osmid', inplace=True)

In [9]:
edges = pd.DataFrame(
    {
        'u': edges_df['u'].astype(int),
        'v': edges_df['v'].astype(int),
        'from': edges_df['u'].astype(int),
        'to': edges_df['v'].astype(int),
        'osmid': edges_df['osmid'].astype(int),
        'length': edges_df['length'].astype(float)
    }
)
edges['key'] = 0
edges['key'] = edges['key'].astype(int)
edges = gpd.GeoDataFrame(data=edges, geometry=edges_df.geometry)
edges.set_index(['u', 'v', 'key'], inplace=True)

In [10]:
net = pdn.Network(
    nodes['lon'],
    nodes['lat'],
    edges['from'],
    edges['to'],
    edges[['length']]
)

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 36958
Setting CH edge vector of size 81667
Range graph removed 82002 edges of 163334
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [7]:
mask = (~edges['to'].isin(nodes['id']))|(~edges['from'].isin(nodes['id']))
mask.sum()

0